In [147]:
import pandas as pd
import numpy as np
import random
import networkx as nx
import bridgeness
from numpy.random import choice, uniform

In [2]:
#FN = "lfr2k_nodes.csv"
FN = "graph.txt.csv"
data = pd.read_csv("graph.txt.csv", delimiter = " ")
#data = pd.read_csv("lfr2_edges.csv", delimiter = "\t")
#data = pd.read_csv("graph2-200nodi-twomoons.txt", header=None, delim_whitespace=True, skipinitialspace=True, usecols=[1,2])#delimiter = "\t"
#data = pd.read_csv(FN, delimiter = "\t")
#print(data)
G = nx.Graph()

for i,s in data.iterrows():
    u = s[0]
    v = s[1]

    G.add_edge(u,v)

#print(G.nodes())

In [3]:
bri = bridgeness.bridgeness_centrality(G)

In [ ]:
#e_bri = bridgeness.edge_bridgeness_centrality(G)

In [ ]:
#bet = bridgeness.betweenness_centrality(G)

In [317]:

"""
net -> adj matr
bri+adj -> tran matr
    in> vec bri, adj mat
    out > tran matr
    
    matr diagonal di vec bri (qualities)
    _tmp = diag*adjmat (prod matriciale)
    norm = diag(sum(temp, sulle righe))
    T = _tmp*(norm^-1) matr trans normalizzata


#1: closed form -> matr inv
    in>  tran_matr + parm mu + labels (y*k)
    out> N*k
    
    alpha = 2/(2+mu)
    op = ( (1-alpha) * eye(N)) / ((eye(N) - alpha * tran_matr))
"""

phi = 100
phi = 3 ###temp
mu = 2.0
mu = 0.01 ##temp
lab0 = 95
lab1 = 190

adj = nx.to_numpy_matrix(G)
quality = (1 / np.exp( np.multiply(bri.values(), phi )))
diag = np.diagflat(quality) #k: v = node, bridgeness
_tmp = np.matmul(diag, adj)
_tmp_sum_on_rows = np.sum(_tmp, axis=0)
_tmp_sum_on_rows_recip = np.reciprocal(_tmp_sum_on_rows)
norm = np.diagflat( _tmp_sum_on_rows_recip )

T = np.matmul( _tmp, norm )

#######

num_nodes = len(G.nodes())
alpha = 2.0/(2.0 + mu)
prob_comm = T 


labels = np.zeros( (num_nodes, 2) )
labels[lab0,0] = 1
labels[lab1,1] = 1

op1 = (1-alpha) * np.eye(num_nodes) 
op2 = ( np.eye(num_nodes) - ( alpha * T))
op = np.dot(op1, op2.I)

probs = np.dot(op, labels)

#bri.items()
#reload(bridgeness)

In [318]:
#print(adj[0])
#print(_tmp[0])
#print(np.sum(adj, axis=0))
#print(_tmp_sum_on_rows)
#print(sum(_tmp[:,0]))
#print(norm)
#print(norm_recip)
#print( sum(T[:,2] ))
#print(T)
#print( (1-alpha) * np.eye(num_nodes) )
#print( np.eye(num_nodes) - ( alpha * T) )
#print(op)
#p0 = probs[:,0]
#p1 = probs[:,1]
#print( probs[p0 >p1] )
#print(probs)
pd.DataFrame.to_csv(pd.DataFrame(probs), 'probs.csv')

In [351]:
#2 RW

#start from lab0(s), lab1(s)


steps = 1000000 #max steps
convcheckfreq = 10000 #check if converged every N steps
conv_thr = 1e-06


n0 = lab0
#n0 = 188
n1 = lab1

labels=[n0,n1]

#visit probability vec
rw_visits = np.zeros( (num_nodes, 2) )
last_rw_visits = np.copy(rw_visits)

nodes = range(num_nodes)

print("Alpha = %.3f" %(alpha))

#COMM0
for s0 in xrange(1,steps+1):

    #start from
    #go back to labeled node? prob = 1-alpha; prob to trans = alpha
    if (uniform() < alpha): #
        trans_probs = T[:,n0].view(np.ndarray).flatten() 
        trans_to = int( choice(nodes, 1, p = trans_probs) )
        n0 = trans_to
    else:
        n0 = lab0

    rw_visits[n0,0] += 1
    
    if (s0 % convcheckfreq == 0):
        diff_rw_visits = ((rw_visits[:,0]/s0 - last_rw_visits[:,0]/s0)**2).sum()
        print("At step %d, diff %.10f" %(s0, diff_rw_visits) )
        if ((diff_rw_visits) < conv_thr):# and np.all(rw_visits != last_rw_visits)):
            print("Converged at <%.10f" % conv_thr)
            break
        last_rw_visits = np.copy(rw_visits)

#print("At step %d, last diff %.5f" % ( s0, diff_rw_visits) )

#COMM1
for s1 in xrange(1,steps+1):

    #start from
    #go back to labeled node? prob = 1-alpha; prob to trans = alpha
    if (uniform() < alpha): #
        #print("Transaction.")
        #probs to trans to node
        trans_probs = T[:,n1].view(np.ndarray).flatten() 
        trans_to = int( choice(nodes, 1, p = trans_probs) )
        #print(trans_to)
        n1 = trans_to
    else:
        #print("Go back.")
        n1 = lab1

    rw_visits[n1,1] += 1
    
    if (s1 % convcheckfreq == 0):
        diff_rw_visits = ((rw_visits[:,1]/s1 - last_rw_visits[:,1]/s1)**2).sum()
        print("At step %d, diff %.10f" %(s1, diff_rw_visits) )
        if ((diff_rw_visits) < conv_thr):# and np.all(rw_visits != last_rw_visits)):
            print("Converged at <%.10f" % conv_thr)
            break
        last_rw_visits = np.copy(rw_visits)

#print("At step %d, last diff %.5f" % ( s1, diff_rw_visits) )




Alpha = 0.995
At step 10000, diff 0.0054439600
At step 20000, diff 0.0013877350
At step 30000, diff 0.0006283689
At step 40000, diff 0.0003838462
At step 50000, diff 0.0002392304
At step 60000, diff 0.0001789167
At step 70000, diff 0.0001342518
At step 80000, diff 0.0000900241
At step 90000, diff 0.0000727296
At step 100000, diff 0.0000634102
At step 110000, diff 0.0000456937
At step 120000, diff 0.0000412819
At step 130000, diff 0.0000383707
At step 140000, diff 0.0000290258
At step 150000, diff 0.0000260314
At step 160000, diff 0.0000284462
At step 170000, diff 0.0000208914
At step 180000, diff 0.0000184643
At step 190000, diff 0.0000174299
At step 200000, diff 0.0000146022
At step 210000, diff 0.0000122014
At step 220000, diff 0.0000118516
At step 230000, diff 0.0000104514
At step 240000, diff 0.0000114272
At step 250000, diff 0.0000089374
At step 260000, diff 0.0000092441
At step 270000, diff 0.0000093526
At step 280000, diff 0.0000069613
At step 290000, diff 0.0000074423
At step 3

In [352]:
#rw_visits[:,0]/steps - last_rw_visits[:,0]/steps
comm_cf = ( (probs[:,0]) > (probs[:,1]) )
comm_rw = ( (rw_visits[:,0]/s0) > (rw_visits[:,1]/s1) )

In [358]:
#print(comm_cf, comm_rw)

In [372]:
#rw_visits[:,0]/s
#sum((rw_visits[:,0]/s0+rw_visits[:,1]/s1))
#rw_visits
#print(np.max(rw_visits[:,0]/s0 - probs[:,0]), np.max(rw_visits[:,1]/s0 - probs[:,1]) )
#print(probs)
print((rw_visits[:,0]/s0, rw_visits[:,1]/s0) )

(array([0.0079    , 0.00689231, 0.00665513, 0.00897051, 0.00608974,
       0.00386795, 0.0049141 , 0.00649487, 0.00656282, 0.00763718,
       0.0059141 , 0.00696282, 0.00613718, 0.00837179, 0.00930769,
       0.00718205, 0.00893462, 0.00414231, 0.00529744, 0.00572179,
       0.00986923, 0.00657949, 0.00641282, 0.00706795, 0.00822179,
       0.00710385, 0.00445769, 0.0071359 , 0.00559487, 0.00695385,
       0.00604487, 0.00654615, 0.00649744, 0.0058859 , 0.00701282,
       0.0056859 , 0.00544744, 0.00661026, 0.00606538, 0.00561538,
       0.00527051, 0.00869744, 0.00891026, 0.00733333, 0.00644359,
       0.00593846, 0.00937051, 0.00618846, 0.00513333, 0.0077    ,
       0.00687692, 0.00598333, 0.00796923, 0.00678077, 0.00809872,
       0.0087641 , 0.00780641, 0.00501538, 0.00564872, 0.00837564,
       0.00548205, 0.00632051, 0.00609744, 0.00478718, 0.00534615,
       0.00416923, 0.01087949, 0.00638718, 0.00476282, 0.00584103,
       0.00758077, 0.00655   , 0.00658333, 0.00755513, 0.0077

In [367]:
#comm_cf